In [21]:
import pandas as pd
import requests

# Load the CSV file into a pandas dataframe
movies_df = pd.read_csv("movie_metadata.csv", nrows=1000)
movies_df

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Color,Andrew Bergman,51.0,117.0,31.0,210.0,Rumer Willis,2000,32800000.0,Comedy|Crime|Drama|Thriller,...,118.0,English,USA,R,40000000.0,1996.0,472.0,4.3,1.85,877
996,Color,Tom Dey,98.0,87.0,9.0,1000.0,Judy Greer,15000,33643461.0,Comedy|Family,...,65.0,English,USA,PG,50000000.0,2010.0,2000.0,4.2,2.35,0
997,Color,Clint Eastwood,315.0,129.0,16000.0,447.0,Jay Mohr,13000,32741596.0,Drama|Fantasy,...,323.0,English,USA,PG-13,50000000.0,2010.0,563.0,6.5,2.35,16000
998,Color,Barbet Schroeder,136.0,115.0,58.0,401.0,Chris Penn,33000,31874869.0,Crime|Mystery|Thriller,...,302.0,English,USA,R,50000000.0,2002.0,455.0,6.1,1.85,0


In [22]:
# Create an empty pandas dataframe to store the API responses
api_responses_df = pd.DataFrame()

# Loop through each movie in the dataframe
for index, row in movies_df.iterrows():
    # Extract the IMDB ID from the movie IMDB link
    imdb_id = row["movie_imdb_link"].split("/")[-2]
    
    # Make an API call to the OMDB API for this movie
    api_url = f"https://www.omdbapi.com/?i={imdb_id}&plot=full&apikey=dc9fb761"
    response = requests.get(api_url)
    
    # Convert the JSON response to a pandas dataframe row
    movie_data = response.json()
    movie_row = pd.DataFrame.from_dict(movie_data, orient="index").T
    
    # Add the new row to the api_responses_df dataframe
    api_responses_df = pd.concat([api_responses_df, movie_row])
    
# Reset the index of the api_responses_df dataframe
api_responses_df = api_responses_df.reset_index(drop=True)

# Merge the api_responses_df dataframe with the original movies_df dataframe
merged_df = pd.concat([movies_df, api_responses_df], axis=1)

# Save the merged dataframe to a new CSV file
merged_df.to_csv("merged_movie_data.csv", index=False)


In [23]:
api_responses_df

,Response,Error,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,...,imdbID,Type,DVD,BoxOffice,Production,Website,Season,Episode,seriesID,totalSeasons
0,False,Invalid API key!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,Invalid API key!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,NaN,Spectre,2015,PG-13,06 Nov 2015,148 min,"Action, Adventure, Thriller",Sam Mendes,"John Logan, Neal Purvis, Robert Wade",...,tt2379713,movie,09 Feb 2016,"$200,074,609",N/A,N/A,NaN,NaN,NaN,NaN
3,False,Invalid API key!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,NaN,Star Wars: Episode VII - The Force Awakens,2015,N/A,18 Dec 2015,34 min,"Documentary, Short",Doug Walker,Doug Walker,...,tt5289954,episode,NaN,NaN,NaN,NaN,5,18,tt5287754,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,True,NaN,Striptease,1996,R,28 Jun 1996,115 min,"Comedy, Crime, Drama",Andrew Bergman,"Carl Hiaasen, Andrew Bergman",...,tt0117765,movie,27 Jul 1999,"$33,109,743",N/A,N/A,NaN,NaN,NaN,NaN
996,False,Invalid API key!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,True,NaN,Hereafter,2010,PG-13,22 Oct 2010,129 min,"Drama, Fantasy, Romance",Clint Eastwood,Peter Morgan,...,tt1212419,movie,15 Mar 2011,"$32,746,941",N/A,N/A,NaN,NaN,NaN,NaN
998,False,Invalid API key!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
merged_df_success_response = merged_df[merged_df["Response"] == "True"]
merged_df_success_response.to_csv("merged_movie_data_success.csv", index=False)
merged_df_success_response

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,imdbID,Type,DVD,BoxOffice,Production,Website,Season,Episode,seriesID,totalSeasons
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000,200074175.0,Action|Adventure|Thriller,...,tt2379713,movie,09 Feb 2016,"$200,074,609",N/A,N/A,NaN,NaN,NaN,NaN
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131,NaN,Documentary,...,tt5289954,episode,NaN,NaN,NaN,NaN,5,18,tt5287754,NaN
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000,336530303.0,Action|Adventure|Romance,...,tt0413300,movie,30 Oct 2007,"$336,530,303",N/A,N/A,NaN,NaN,NaN,NaN
8,Color,Joss Whedon,635.0,141.0,0.0,19000.0,Robert Downey Jr.,26000,458991599.0,Action|Adventure|Sci-Fi,...,tt2395427,movie,02 Oct 2015,"$459,005,868",N/A,N/A,NaN,NaN,NaN,NaN
9,Color,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000,301956980.0,Adventure|Family|Fantasy|Mystery,...,tt0417741,movie,08 Dec 2009,"$302,334,374",N/A,N/A,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,Color,James McTeigue,235.0,99.0,160.0,72.0,Ben Miles,330,38105077.0,Action|Crime|Thriller,...,tt1186367,movie,16 Mar 2010,"$38,122,883",N/A,N/A,NaN,NaN,NaN,NaN
994,Color,NaN,14.0,105.0,NaN,5.0,Bruce Alexander,325,NaN,Crime|Drama|Mystery,...,tt0108967,series,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
995,Color,Andrew Bergman,51.0,117.0,31.0,210.0,Rumer Willis,2000,32800000.0,Comedy|Crime|Drama|Thriller,...,tt0117765,movie,27 Jul 1999,"$33,109,743",N/A,N/A,NaN,NaN,NaN,NaN
997,Color,Clint Eastwood,315.0,129.0,16000.0,447.0,Jay Mohr,13000,32741596.0,Drama|Fantasy,...,tt1212419,movie,15 Mar 2011,"$32,746,941",N/A,N/A,NaN,NaN,NaN,NaN
